<a href="https://colab.research.google.com/github/emkalan/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Nam(s):
#### Date:

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-13 <class 'datetime.date'>
Today is: 2025-01-13


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv('API_KEY.env')  # Load .env file
API_KEY = os.getenv('API_KEY')

if API_KEY:
    print("API Key loaded successfully!")
else:
    print("Failed to load API Key.")


# get today's date (as a string)
today = str(datetime.date.today())

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

API Key loaded successfully!
17 Near Earth Objects found for 2025-01-13
[{'absolute_magnitude_h': 16.48,
  'close_approach_data': [{'close_approach_date': '2025-01-13',
                           'close_approach_date_full': '2025-Jan-13 01:06',
                           'epoch_date_close_approach': 1736730360000,
                           'miss_distance': {'astronomical': '0.2580355174',
                                             'kilometers': '38601563.787387938',
                                             'lunar': '100.3758162686',
                                             'miles': '23985899.4965440244'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '56146.0991462165',
                                                 'kilometers_per_second': '15.5961386517',
                                                 'miles_per_hour': '34887.0025500477'}}],
  'estimated_diameter': {'feet': {'estimated_diame

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

0 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [ ]:
if hazards:
  id = hazards[0]['id']
  url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
  response = requests.request("GET", url, headers={}, data={})
  data = json.loads(response.text)
  pprint(data)
else:
  print('No hazardous asteroids found')

No hazardous asteroids found


## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [ ]:
# Ex. 1 your code here
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

0 potentially hazardous asteroids identified.




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [ ]:
# Ex. 2 your code here
def closest_approach_to_earth(data):
    """
    Find the closest approach to Earth from a list of close approach data.

    Parameters:
        data (list): A list of dictionaries, each containing:
            - 'close_approach_date' (str): The date of the close approach.
            - 'orbiting_body' (str): The celestial body being orbited.
            - 'miss_distance' (dict): A dictionary with keys like 'kilometers'.

    Returns:
        tuple: A 2-tuple (date, miss_km) of the closest approach to Earth.
    """
    closest_date = None
    closest_miss_distance = float('inf')

    for entry in data:
        if entry['orbiting_body'] == 'Earth':
            miss_km = float(entry['miss_distance']['kilometers'])
            if miss_km < closest_miss_distance:
                closest_miss_distance = miss_km
                closest_date = entry['close_approach_date']

    return closest_date, closest_miss_distance

# Unit Test
def test_closest_approach_to_earth():
  """ Simple unit test for the closest_approach_to_earth function. """
    test_data = [
        {
            'close_approach_date': '2025-01-01',
            'orbiting_body': 'Earth',
            'miss_distance': {'kilometers': '1000000'}
        },
        {
            'close_approach_date': '2025-01-05',
            'orbiting_body': 'Earth',
            'miss_distance': {'kilometers': '750000'}
        },
        {
            'close_approach_date': '2025-01-03',
            'orbiting_body': 'Mars',
            'miss_distance': {'kilometers': '500000'}
        },
        {
            'close_approach_date': '2025-01-10',
            'orbiting_body': 'Earth',
            'miss_distance': {'kilometers': '2000000'}
        }
    ]

    result = closest_approach_to_earth(test_data)
    assert result == ('2025-01-05', 750000.0), f"Unexpected result: {result}"

test_closest_approach_to_earth()


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
import requests
from datetime import date, timedelta

BASE_URL = "https://api.nasa.gov/neo/rest/v1"

def fetch_neo_list():
  """Fetches the list of NEO's for this week."""
    end_date = date.today()
    start_date = end_date - timedelta(days=7)
    url = f"{BASE_URL}/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}"

    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return data["near_earth_objects"]

def find_closest_approach(neo_data):
  """Finds the closest NEO from this week's data."""
    today_str = str(date.today())
    closest_neo = min(
        [
            {
                "name": neo["name"],
                "date": approach["close_approach_date"],
                "distance_km": float(approach["miss_distance"]["kilometers"]),
            }
            for date_key, neos in neo_data.items()
            for neo in neos
            for approach in neo["close_approach_data"]
            if approach["close_approach_date"] >= today_str
        ],
        key=lambda x: x["distance_km"],
        default=None
    )
    return closest_neo

def main():
    try:
        neo_data = fetch_neo_list()
        closest_neo = find_closest_approach(neo_data)

        if closest_neo:
            print(f"The closest NEO this week is '{closest_neo['name']}'.")
            print(f"It will approach Earth on {closest_neo['date']},")
            print(f"coming as close as {closest_neo['distance_km']:.2f} kilometers. Groovy!")
        else:
            print("No close approaches found for this week.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}") # THIS ONE WAS REALLY IMPORTANT  turns out you REALLY need list comprehension. i kept getting too many requests errors. 429 eror.

if __name__ == "__main__":
    main()


The closest NEO this week is '(2025 AX2)'.
It will approach Earth on 2025-01-13,
coming as close as 3151350.09 kilometers. Groovy!
